In [ ]:
import numpy as np

# 0 = no wind, 1 = 1 up, 2 = 2 up
windy_gw = np.zeros((7, 10), dtype=np.int8)

windy_gw[0:7, 3:6] = 1
windy_gw[0:7, 8] = 1

windy_gw[0:7, 6:8] = 2

start_location = np.array([3, 0])

goal = np.array([3, 7])